In [10]:
from os import stat_result
from casadi import *
import numpy as np
import tensorflow as tf
from casadi import Sparsity


class Neural_Network(Callback):
    def __init__(self, name, model, opts={}):
        self.model = model
        Callback.__init__(self)
        self.construct(name, opts)

    # Number of inputs and outputs
    def get_n_in(self): return 1

    def get_n_out(self): return 1

    def get_sparsity_in(self, i):
        return Sparsity.dense(10, 1)

    def get_sparsity_out(self, i):
        return Sparsity.dense(5, 1)

    # Evaluate numerically
    def eval(self, arg):
        model = self.model
        s = np.array(arg[0])
        s = s.astype('float32')
        f = model.predict(s.T)
        return [f[0]]



In [ ]:

def MPC(s, model, xref=[1, 0, 0]):
    opti = casadi.Opti()

    nn = Neural_Network('netwerk', model, opts={"enable_fd": True})

    state = MX.sym('state', 3)
    action = MX.sym('action')
    input = vertcat(state, action)
    f = Function('f', [state, action], [nn(input)])

    #time horizon
    N = 100

    x = opti.variable(3, N + 1)
    u = opti.variable(1, N)

    x_ref = opti.parameter(3, N + 1)
    p = opti.parameter(3, 1)

    ref = np.zeros((3, N + 1))
    for i in range(N + 1):
        ref[0, i] = xref[0]
        ref[1, i] = xref[1]
        ref[2, i] = xref[2]

    Q = MX.eye(3)  # needs to be optimised
    Q[0, 0] = 100
    Q[1, 1] = 5
    R = MX.zeros(1, 1)  # MX.eye(1)

    def cost(x, u, x_ref):
        s = 0
        for n in range(N):
            s += mtimes(mtimes((x[:, n] - x_ref[:, n]).T, Q), (x[:, n] - x_ref[:, n]))
            s += mtimes(mtimes(u[:, n].T, R), u[:, n])
        s += mtimes(mtimes((x[:, N] - x_ref[:, N]).T, Q), (x[:, N] - x_ref[:, N]))
        return s

    opti.minimize(cost(x, u, x_ref))

    for k in range(N):
        opti.subject_to(x[:, k + 1] == f(x[:, k], u[:, k]))

    opti.subject_to(x[:, 0] == p)
    opti.subject_to(u <= 2)
    opti.subject_to(u >= -2)
    opti.solver('sqpmethod', {'qpsol': 'qrqp'})

    opti.set_value(p, s)
    opti.set_value(x_ref, ref)

    sol = opti.solve()
    return sol.value(u)

In [11]:
model=tf.keras.models.load_model('output/disk.h5')
nn = Neural_Network('netwerk', model, opts={"enable_fd": True})

state = MX.sym('state', 8)
control = MX.sym('action',2)
input = vertcat(state, control)
f = Function('f', [state, control], [nn(input)])

N = 50
nx = 8
nu = 2
dt = 0.05
x0 = np.pi*np.array([1, 1, -1.5, 1, -1, 1, 0, 0]).reshape(-1,1)

option = {}
option['max_iter']=10000

#option['print_level']=0

opti = casadi.Opti()
x = opti.variable(8,N+1)
u = opti.variable(2,N+1)

phi_1= x[0,:]
phi_2= x[1,:]
phi_3= x[2,:]
dphi_1= x[3,:]
dphi_2= x[4,:]
dphi_3= x[5,:]
phi_1_m= x[6,:]
phi_2_m= x[7,:]

phi_m_1_set = u[0,:]
phi_m_2_set = u[1,:]

for k in range(N):
    opti.subject_to(x[0:3, k + 1] == x[0:3, k] + dt * x[3:6, k])
    opti.subject_to(x[3:8, k + 1] == x[3:8, k] + dt * f(x[:, k], u[:, k]))


opti.minimize(casadi.dot(phi_1,phi_1)+casadi.dot(phi_2,phi_2)+casadi.dot(phi_3,phi_3)
              +0.001*casadi.dot(phi_m_1_set,phi_m_1_set)+0.1*casadi.dot(phi_m_2_set,phi_m_2_set)
              +0.001*casadi.dot(phi_1_m,phi_1_m)+0.1*casadi.dot(phi_2_m,phi_2_m))


opti.subject_to(x[:,0]==x0)

opti.subject_to(opti.bounded(-2*casadi.pi,phi_1,2*casadi.pi))
opti.subject_to(opti.bounded(-2*casadi.pi,phi_2,2*casadi.pi))
opti.subject_to(opti.bounded(-2*casadi.pi,phi_3,2*casadi.pi))

opti.subject_to(opti.bounded(-2*casadi.pi,phi_1_m,2*casadi.pi))
opti.subject_to(opti.bounded(-2*casadi.pi,phi_1_m,2*casadi.pi))


opti.subject_to(opti.bounded(-2*casadi.pi,phi_m_1_set,2*casadi.pi))
opti.subject_to(opti.bounded(-2*casadi.pi,phi_m_2_set,2*casadi.pi))



opti.solver("ipopt",{},option)

try:
    sol = opti.solve()

except Exception as e:
    print(e)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     3208
Number of nonzeros in inequality constraint Jacobian.:      357
Number of nonzeros in Lagrangian Hessian.............:     2757

1/1 [==============================] - 0s 12ms/step
Total number of variables............................:      510
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables w

CasADi - 2022-10-10 17:02:00 WARNING("solver:nlp_hess_l failed:Error in Function::operator() for 'nlp_hess_l' [MXFunction] at .../casadi/core/function.cpp:1368:
Error in Function::operator() for 'fwd10_adj1_f' [MXFunction] at .../casadi/core/function.cpp:1368:
Error in Function::operator() for 'fwd10_jac_wrap_netwerk' [MXFunction] at .../casadi/core/function.cpp:1368:
Error in Function::operator() for 'fwd10_fwd10_netwerk' [CentralDiff] at .../casadi/core/function.cpp:1368:
Error in Function::operator() for 'fwd10_netwerk' [CentralDiff] at .../casadi/core/function.cpp:1368:
Error in Function::operator() for 'netwerk' [CallbackInternal] at .../casadi/core/function.cpp:1368:
.../casadi/core/function_internal.cpp:3366: Failed to evaluate 'eval_dm' for netwerk:
.../casadi/core/callback_internal.cpp:122: Error calling "eval" for object netwerk:
KeyboardInterrupt") [.../casadi/core/oracle_function.cpp:223]
